# Market Trend Analysis on Apple iOS and Play Store Apps
In this project, we analysed data to help app developers understand what type of apps are likely to attract more users. Our aim is to develop apps that can sustain both in the iOS and Play store world.

We are going to focus mainly on building apps that are free to download and install and the main source of revenue consists of in-app ads. This means the revenue for any given app is mostly influenced by the number of users who use an app, the more users that see and engage with the ads, the better.

## Opening, reading and exploring data
We had outlined that our aim is to help our developers understand what type of apps are likely to attract more users on Google Play and the App Store. To do this, we'll need to collect and analyze data about mobile apps available on Google Play and the App Store.

Firstly, we are going to create a function to open and read the data available. In the first function(csv_as_list), we will return a **list of lists**. Here, each row(or a datapoint) is in the form of a list that gets stored in an outer list, hence the format **list of lists**.

In [53]:
def csv_as_list(file_name):
    open_file = open(file_name,encoding = 'UTF8') #To avoid UnicodeDecode Error
    from csv import reader
    read_file = reader(open_file)
    data_in_list = list(read_file) #creating a list of lists
    return data_in_list

app_store_data = csv_as_list('AppleStore.csv')
play_store_data = csv_as_list('googleplaystore.csv')

A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store was used for analysis and this data was collected in July 2017.

A [data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play was used for analysis purposes and this data was collected in August 2018.

Next, let's make use of another function to explore the dataset in a more readable way. We can also provide an additional argument that fetches us the number of rows and columns in the dataset.

In [54]:
def explore_data(dataset, start, end, rows_and_columns = False):
    data_slice = dataset[start:end]   
    for row in data_slice:
        print(row)
        print('\n') #adding a new line after each row
    if rows_and_columns:
        print('Number of rows: ', len(dataset))
        print('Number of columns: ', len(dataset[0]), '\n')


Let us explore the appstore data first:

In [55]:
header = app_store_data[0]
print(header,'\n')
explore_data(app_store_data, 1, 3, True)


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows:  7198
Number of columns:  16 



App store dataset has 7198 apps each with 16 columns of data. Some of the attributes that look interesting are: 'size_bytes', price, rating_count_tot, prime_genre to name a few.

Now let's take a look at the playstore dataset:

In [56]:
header = play_store_data[0]
print(header,'\n')
explore_data(play_store_data, 1, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows:  10842
Number of columns:  13 



Play store dataset has 10842 rows with 13 columns of data each. Few attributes that look useful at first glance are: 'Category', 'Rating', 'Installs', 'Genres' to name a few. If some of the attribute names are not descriptive enough, you can refer to this [documentation](https://www.kaggle.com/lava18/google-play-store-apps).

## Data Cleaning
### Part 1: Deleting Incorrect Data
The process of preparing our data for analysis is called **data cleaning**. Data cleaning is done before the analysis. It includes removing or correcting wrong data, removing duplicate data, and modifying the data to fit the purpose of our analysis.

Let's start off by writing a block of code which identifies irregularities or wrong data in the dataset(if any) by checking the **length** of each **datapoint** with the length of the **header**. If the lenghts are different, our code intercepts that irregularity and prints that row along with it's index number.

In [57]:
header = play_store_data[0]
print(header,'\n', "Header row length:", len(header))

for row in play_store_data:
    header_length = len(header)
    row_length = len(row)
    if row_length != header_length: #checking for possible incorrect data points
        print(row)
        print('Row number: ',play_store_data.index(row)) #extracting the incorrect row and corresponding index number
        print("Row length:", len(row))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 
 Header row length: 13
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
Row number:  10473
Row length: 12


We observe that the play store data set contains an irregularity in row number 10473. This took place due to the absence of data under the 'Category' attribute which caused the rest of the data to shift by one unit. As a result, we delete this row.

In [58]:
print("Number of datapoints with irregularity:",len(play_store_data))
#del play_store_data[10473] #If run more than once, will result in deletion of multiple rows, hence commenting out
print("Number of datapoints without irregularity",len(play_store_data))

Number of datapoints with irregularity: 10842
Number of datapoints without irregularity 10842


### Part 2: Removing Duplicate Entries
In the last step, we started the data cleaning process and deleted a row with incorrect data from the Google Play data set. If you explore the Google Play data set long enough or look at the [discussions section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), you'll notice some apps have duplicate entries. For instance, Instagram has four entries:

In [59]:
for row in play_store_data[1:]: #excluding header row
    app_name = row[0]
    if app_name == 'Instagram':
        print(row)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In total, there are 1,181 cases where an app occurs more than once:

In [60]:
unique_apps = []
duplicate_apps = []

for row in play_store_data:
    name = row[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Number of duplicate apps: ', len(duplicate_apps))
print('Some of the duplicate apps are:',duplicate_apps[0:5])

Number of duplicate apps:  1181
Some of the duplicate apps are: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. If you examine the rows we printed for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the ***number of reviews***. The different numbers show the data was collected at different times.

We can use this information to build a criterion for removing the duplicates. The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.

In [61]:
reviews_max = {}

for row in play_store_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if (name in reviews_max) and (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print(reviews_max)
        

ValueError: could not convert string to float: '3.0M'

To remove the duplicates, we:

* Created a dictionary(reviews_max), where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.
* Using the information stored in the dictionary above,we created a new data set below, which will have only one entry per app (and for each app, we'll only select the entry with the highest number of reviews).

Let us create two new lists, "android_clean" which will store our new cleaned data set and "already_added" which will just store app names.

In [ ]:
android_clean = []
already_added = []

for row in play_store_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(row)
        already_added.append(name)   

Now, let us explore our newly created dataset

In [ ]:
explore_data(android_clean,0,3,True)

### Part 3: Removing Non-English Apps
Our aim was to use only English for the apps we develop at our company and we'd like to analyze only the apps that are directed toward an English-speaking audience. However, if we explore the data long enough, we'll find that both data sets have apps with names that suggest they are not directed toward an English-speaking audience. For example:

In [ ]:
#Non English Apps in App Store
print(app_store_data[814][1])
print(app_store_data[6732][1])

#Non English Apps in Play Store
print(android_clean[4412][0])
print(android_clean[7940][0])


We're not interested in keeping these apps, so we'll remove them. One way to go about this is to remove each app with a name containing a symbol that is not commonly used in English text. English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name.

We built this function below, and we use the built-in ord() function to find out the corresponding encoding number of each character.

In [ ]:
def is_english(string):
    for letter in string:
        if ord(letter) > 127:
            return False
    return True

print(is_english('Facebook'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

The above function works well in most of the cases but, it couldn't classify certain app names such as 'Instachat 😜', 'Docs To Go™ Free Office Suite' as **True**. This is because emojis and characters like '™' fall outside the ASCII range and have corresponding numbers over 127.

In [ ]:
print(ord('™'))
print(ord('😜'))
print(is_english('Instachat 😜'))
print(is_english('Docs To Go™ Free Office Suite'))

To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with **up to three emoji or other special characters** will still be labeled as English.

In [ ]:
def is_english(string):
    counter = 0
    for letter in string:
        code = ord(letter)
        if code > 127: 
            counter += 1
    if counter > 3:
        return False
    else:
        return True
    

print(is_english('Instachat 😜'))
print(is_english('Docs To Go™ Free Office Suite'))

Now, let us filter the datasets only for English apps.

In [ ]:
english_app_store = []
english_play_store = []

for row in app_store_data[1:]:
    name = row[1]
    result = is_english(name)
    if result == 1:
        english_app_store.append(row)
    else:
        continue

for row in android_clean:
    name = row[0]
    result = is_english(name)
    if result == 1:
        english_play_store.append(row)
    else:
        continue

explore_data(english_app_store, 0, 3, True)
print('\n')
explore_data(english_play_store, 0, 3, True)

### Part 3: Isolating the Free Apps
As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, we'll need to isolate only the free apps for our analysis.

In [ ]:
final_app_store = []
final_play_store = []

for row in english_app_store:
    price = float(row[4])
    if price == 0:
        final_app_store.append(row)
    else:
        continue
        
for row in english_play_store:
    price = row[6]
    if price == 'Free':
        final_play_store.append(row)
    else:
        continue
        
print(len(final_app_store ))
print(len(final_play_store ))

So far, we spent a good amount of time on cleaning data and:

* Removed inaccurate data
* Removed duplicate app entries
* Removed non-English apps
* Isolated the free apps

We are now left with 3222 and 8863 data points of App Store and Play store respectively.

## Most Common Apps by Genre
### Part One

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

* Build a minimal Android version of the app and add it to Google Play.
* If the app has a good response from users, we develop it further.
* If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets.

### Part Two
We'll build two functions we can use to analyze the frequency tables:

- One function(freq_table) to generate frequency tables that show percentages of an attribute.
- Another function(display_table) we can use to display the percentages in a descending order.

In [ ]:
def freq_table(dataset, index):
    tot = len(dataset)
    table = {}
    for row in dataset:
        attribute = row[index]
        if attribute in table:
            table[attribute] += 1
        else:
            table[attribute] = 1
    table_percentage = {}
    for key in table:
        percentage = (table[key]/tot)*100
        table_percentage[key] = percentage
    return table_percentage

def display_table(dataset, index):
    frequencies = freq_table(dataset, index)
    reversed_pairs = []
    for key, val in frequencies.items():
        pair = (val, key) #creating a reversed tuple in the form of a (value, key) pair
        reversed_pairs.append(pair)
        
    reversed_pairs = sorted(reversed_pairs, reverse = True) 
    
    for val, key in reversed_pairs:
        print (key,':',val)

### Part Three
Let us now create a few frequency tables of certain attributes from both the datasets.

In [ ]:
display_table(final_app_store,-5) #prime_genre

In [ ]:
display_table(final_play_store,1) #By Category

In [ ]:
display_table(final_play_store, -4) #By Genre

From the above three tables, we observe that:
* Most of the apps are designed for gaming and entertainment in the app store.
* Uniform distribution of different types of apps in the play store.

At this stage, we can't make any firm conclusions, as the number of gaming apps available on the app store does not necessarily mean more amount of people use and like those apps. Hence, we need to make use of various other attributes to make further deductions.

## Most popular apps by genre on the App store
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [ ]:
app_store_genre = freq_table(final_app_store, -5) #frequency table by genre

for genre in app_store_genre:
    total = 0
    tot_ratings = 0
    len_genre = 0
    for row in final_app_store:
        genre_app = row[-5]
        num_ratings = float(row[5])
        user_rating = float(row[13])
        if genre_app == genre:
            total += num_ratings
            tot_ratings += user_rating
            len_genre += 1
    print(genre,':',round(total/len_genre))


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [ ]:
for row in final_app_store:
    if row[-5] == 'Navigation':
        print(row[1],float(row[5]), float(row[7]))

The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [ ]:
for row in final_app_store:
    if row[-5] == 'Reference':
        print(row[1],float(row[5]))


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

## Most Popular Apps by Genre on Google Play
For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [ ]:
display_table(final_play_store, 5)

One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [ ]:
play_store_category = freq_table(final_play_store,1)

for category in play_store_category:
    total = 0
    len_category = 0
    for row in play_store_data[1:]:
        app_category = row[1]
        n_installs = row[5]
        n_installs = n_installs.replace('+','')
        n_installs = n_installs.replace(',','')
        n_installs = float(n_installs)
        if category == app_category:
            total += n_installs
            len_category += 1
    print(category,'average:',total/len_category)

In [ ]:
for row in final_play_store:
    if row[1] == 'COMMUNICATION' and (row[5] == '1,000,000,000+'
                                      or row[5] == '500,000,000+'
                                      or row[5] == '100,000,000+'):
        print(row[0], ':', row[5])

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [ ]:
under_100_m = []

for app in final_play_store:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [ ]:
for app in final_play_store:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [ ]:
for row in final_play_store:
    if row[1] == 'BOOKS_AND_REFERENCE' and (row[5] == '1,000,000,000+'
                                            or row[5] == '500,000,000+'
                                            or row[5] == '100,000,000+'):
        print(row[0], ':', row[5])

However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [ ]:
for row in final_play_store:
    if row[1] == 'BOOKS_AND_REFERENCE' and (row[5] == '1,000,000+'
                                            or row[5] == '5,000,000+'
                                            or row[5] == '10,000,000+'
                                            or row[5] == '50,000,000+'):
        print(row[0], ':', row[5])


This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.